In [1]:
import numpy as np
import pandas as pd

from modules.google_sheets_manager import GoogleSheetsManager
from modules.simulation import evaluate_trades
from modules.download import download_data
from modules.indicators import *

In [2]:
gs_url = "https://docs.google.com/spreadsheets/d/16HZfFIu37ZG7kRgLDI9SqS5xPhb3pn3iY2ubOymVseU/edit#gid=1171524967"

gs_manager = GoogleSheetsManager(gs_url)

## Read Trades

In [3]:
read_trades_df = gs_manager.read_trades("Test")
read_trades_df.head()

/Users/ivanosipchyk/dev/investing/ema-crossover/modules/google_sheets_manager.py:152: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[col] = pd.to_numeric(df[col], errors="ignore")


,Date,Symbol,Entry Price,Shares,Position Size,Stop Loss,Risk,Risk per Share,BE Price,BE Level,Position Left,Realized,Unrealized,TP Reached,Stop Rule,Offset,Max TP,Skip TP,TP Coef
0,2025-09-24,COR,302.93,0.33,100,290.00,4.27,12.93,302.93,315.86,1,NaN,NaN,NaN,crossover,0,5,1,2
1,2025-09-24,HIG,132.46,0.75,100,130.00,1.86,2.46,132.46,134.92,1,NaN,NaN,NaN,crossover,0,5,1,2
2,2025-09-24,NJR,47.34,2.11,100,46.32,2.15,1.02,47.34,48.36,1,NaN,NaN,NaN,crossover,0,5,1,2
3,2025-09-24,AROC,26.00,3.85,100,24.56,5.54,1.44,26.00,27.44,1,NaN,NaN,NaN,crossover,0,5,1,2
4,2025-09-25,CB,279.76,0.36,100,275.50,1.52,4.26,279.76,284.02,1,NaN,NaN,NaN,crossover,0,5,1,2


### PASS

## Write Potential Entries

In [4]:
today = np.datetime64("today")
mock_potential_entries = [(["TEST", "TICKER"], ["FILTER"])]
gs_manager.write_potential_entries(today, mock_potential_entries)

### PASS

## Read Previous Trading Day Entries

In [6]:
prev_day_entries = gs_manager.read_previous_trading_day_entries()
print(prev_day_entries)

['ASB', 'BOX', 'BP', 'BZ', 'COLB', 'FHN', 'HSII', 'HWC', 'MAR', 'PFS', 'PHG', 'SANM', 'TNL', 'XYZ']


### PASS

## Update Trades

In [14]:
columns = [
    "Date", "Symbol", "Entry Price", "Shares", "Position Size", "Stop Loss",
    "Risk", "Risk per Share", "BE Price", "BE Level", "Position Left",
    "Realized", "Unrealized", "TP Reached", "Stop Rule", "Offset",
    "Max TP", "Skip TP", "TP Coef"
]

row = [
    "2025-10-06 0:00:00", "HSII", 58.15, 0.64, 37.2, 50.33, 5, 7.82,
    58.15, 65.97, 1, "", "", "", "crossover", -20, 4, 1, 2
]

mock_new_trades = pd.DataFrame([row], columns=columns)
mock_new_trades["Date"] = pd.to_datetime(mock_new_trades["Date"])
mock_new_trades.head()

,Date,Symbol,Entry Price,Shares,Position Size,Stop Loss,Risk,Risk per Share,BE Price,BE Level,Position Left,Realized,Unrealized,TP Reached,Stop Rule,Offset,Max TP,Skip TP,TP Coef
0,2025-10-06,HSII,58.15,0.64,37.2,50.33,5,7.82,58.15,65.97,1,,,,crossover,-20,4,1,2


In [15]:
row = [
    "2025-10-06 0:00:00", "SANM", 160.25, 0.13, 21.52, 123.02,
    5, 37.23, 160.25, 197.48, 1, "", "", "", "crossover", -20,
    4, 1, 2
]

mock_new_trades_2 = pd.DataFrame([row], columns=columns)
mock_new_trades_2["Date"] = pd.to_datetime(mock_new_trades_2["Date"])
mock_new_trades_2.head()

,Date,Symbol,Entry Price,Shares,Position Size,Stop Loss,Risk,Risk per Share,BE Price,BE Level,Position Left,Realized,Unrealized,TP Reached,Stop Rule,Offset,Max TP,Skip TP,TP Coef
0,2025-10-06,SANM,160.25,0.13,21.52,123.02,5,37.23,160.25,197.48,1,,,,crossover,-20,4,1,2


In [16]:
mock_historical_data = download_data(['HSII', 'SANM'], '200d', '1d')
mock_historical_data_labeled = apply_to_dict(mock_historical_data, add_indicators)

Processing symbols:   0%|          | 0/2 [00:00<?, ?it/s]

### UC1: No Existing Tab

In [10]:
gs_manager.update_trades(
    mock_new_trades, 'Non Exisitng Tab', 
    evaluate_trades, mock_historical_data_labeled
)

### UC2: Empty Tab Exists

In [13]:
gs_manager.update_trades(
    mock_new_trades, 'Non Exisitng Tab', 
    evaluate_trades, mock_historical_data_labeled
)

⚠️ No trade data found.


### UC3: Non-empty Tab Exists

In [17]:
gs_manager.update_trades(
    mock_new_trades_2, 'Non Exisitng Tab', 
    evaluate_trades, mock_historical_data_labeled
)

/Users/ivanosipchyk/dev/investing/ema-crossover/modules/google_sheets_manager.py:152: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[col] = pd.to_numeric(df[col], errors="ignore")


### PASS